In [64]:
from pocket_cube.cube import Cube, Move
from pocket_cube.tests import case1, case2, case3, case4
from time import process_time_ns
from random import choice
from heapq import heappop, heappush
from math import sqrt, log
from collections import deque
import matplotlib.pyplot as plt
import tracemalloc
import numpy as np
%matplotlib notebook

# Environment Setup Configuration for Algorithm Comparison & Analysis

In [65]:
unscrambled_cube = Cube(scrambled=False)
cube_tests = [Cube(case1), Cube(case2), Cube(case3), Cube(case4)]
target = [0, 0, 0 ,0, 1, 1, 1, 1, 2, 2, 2, 2 ,3 ,3 ,3, 3, 4, 4, 4, 4, 5, 5, 5 ,5]
plt.rcParams.update({'figure.max_open_warning': 0})

dummy_cube = Cube(scrambled=False)
N = 'N'
Q = 'Q'
PARENT = 'parent'
ACTIONS = 'actions'

In [66]:
def euclid_dist_cube(cube_state) -> float:
    dist = 0
    for elem in range(6):
        for i in range(elem * 4, (elem + 1) * 4):
                dist += (cube_state[i] - target[i])**2

    return sqrt(dist)

# h1 heuristic (which is admissible) from the homework statement
def rotation_cube(cube_state) -> int:
    dist = 0
    for elem in range(6):
        for i in range(elem * 4, (elem + 1) * 4):
            if (cube_state[i] == 0 and elem == 2) or \
               (cube_state[i] == 5 and elem == 4) or \
               (cube_state[i] == 1 and elem == 3):
                dist += 2
            elif cube_state[i] != elem:
                dist += 1
            else:
                dist += 0
    return dist / 4

# A* Algorithm for Pocket Cube Problem

In [67]:
def get_neighbours(cube : Cube) -> dict:
    avl_moves = {Move.R: cube.move(Move.R), Move.U: cube.move(Move.U), Move.F: cube.move(Move.F),
                 Move.Rp : cube.move(Move.Rp), Move.Up : cube.move(Move.Up), Move.Fp : cube.move(Move.Fp)}
    return avl_moves

def astar(start : Cube, end : Cube, h) -> (list, list):
    frontier = []
    list_moves = []
    heappush(frontier, (h(start.state), start))
    discovered = {start.hash(): (None, 0, "")}
    while frontier:
        _, curr_node = heappop(frontier)
        curr_g_cost = discovered[curr_node.hash()][1]

        if curr_node.hash() == end.hash():
            break

        for (neigh_move, neigh_node) in get_neighbours(curr_node).items():
            neigh_g_cost = curr_g_cost + 1

            if neigh_node.hash() not in discovered or neigh_g_cost < discovered[neigh_node.hash()][1]:
                discovered[neigh_node.hash()] = (curr_node, neigh_g_cost, neigh_move)
                heappush(frontier, (neigh_g_cost + h(neigh_node.state), neigh_node))

    path = []
    crt_node = end
    while crt_node:
        list_moves.append(discovered[crt_node.hash()][2])
        path.append(crt_node)
        crt_node = discovered[crt_node.hash()][0]

    path.reverse()
    list_moves.reverse()
    list_moves = list_moves[1:]

    return len(path), len(discovered), list_moves

In [5]:
test_num = 1

dict_times_astar = {}
dict_states_nums_astar = {}
dict_sol_lens_astar = {}
dict_mem_astar = {}

for elem in cube_tests:
    print(f"Execution for case{test_num}")

    tracemalloc.start()
    len_solution, num_of_states, moves_astar = astar(elem.clone(), unscrambled_cube, rotation_cube)
    memory = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    start_time = process_time_ns()
    astar(elem.clone(), unscrambled_cube, rotation_cube)
    end_time = process_time_ns()
    running_time = end_time - start_time

    print(f"Running time: {running_time * 10**(-9)} s")
    print(f"Length of path to solution: {len_solution}")
    print(f"Number of discovered states: {num_of_states}")
    print(f"Used memory: {memory / 2**10} KB")

    dict_times_astar.update({(test_num, rotation_cube) : running_time * 10**(-9)})
    dict_sol_lens_astar.update({(test_num, rotation_cube) : len_solution})
    dict_states_nums_astar.update({(test_num, rotation_cube) : num_of_states})
    dict_mem_astar.update({(test_num, rotation_cube) : memory / 2**10})
    test_num += 1

    print("\n")

Execution for case1
Running time: 0.015625 s
Length of path to solution: 6
Number of discovered states: 186
Used memory: 148.0546875 KB


Execution for case2
Running time: 0.171875 s
Length of path to solution: 8
Number of discovered states: 1618
Used memory: 1228.8759765625 KB


Execution for case3
Running time: 3.125 s
Length of path to solution: 10
Number of discovered states: 25118
Used memory: 20088.359375 KB


Execution for case4
Running time: 35.890625 s
Length of path to solution: 12
Number of discovered states: 267400
Used memory: 211272.81640625 KB


# Bidirectional BFS Algorithm for Pocket Cube Problem

In [68]:
def bidirectional_bfs_path(position, parents) -> (list, list):
    path = []
    list_moves = []
    while True:
        pair_info = parents[position.hash()]

        if pair_info is None:
          path.reverse()
          list_moves.reverse()
          return path, list_moves

        path.append(pair_info[1].state)
        if pair_info[0] == Move.Rp:
            list_moves.append(Move.R)
        elif pair_info[0] == Move.Up:
            list_moves.append(Move.U)
        elif pair_info[0] == Move.Fp:
            list_moves.append(Move.F)
        else:
            list_moves.append(pair_info[0])

        position = pair_info[1]

def bidirectional_bfs(start : Cube, end : Cube) -> (list, list):

    result = []
    list_moves = []
    list_moves_forward = []
    list_moves_backward = []
    forward_path = []
    backward_path = []
    cube = Cube(scrambled=False)

    if start.hash() == end.hash(): 
        return result

    forward_parents = {start.hash(): None}
    backward_parents = {end.hash(): None}

    forward_moves = {Move.R: Move.R, Move.U: Move.U, Move.F: Move.F}
    backward_moves = {Move.Rp: Move.Rp, Move.Up: Move.Up, Move.Fp: Move.Fp}

    forward = (forward_moves, forward_parents, backward_parents)
    backward = (backward_moves, backward_parents, forward_parents)

    queue = deque([(start, forward), (end, backward), None])
    discovered_states = {start : forward, end : backward}
    while not result:
        node_info_pair = queue.popleft()
        if node_info_pair is None:
            node_info_pair = queue.popleft()
        curr_state = node_info_pair[0]
        moves, parents, other_parents = node_info_pair[1]
        for move in moves:
            next_position = curr_state.move(move)
            if next_position.hash() in parents:
                continue
            parents[next_position.hash()] = (moves[move], curr_state)
            queue.append((next_position, node_info_pair[1]))
            discovered_states.update({next_position : node_info_pair[1]})
            if next_position.hash() in other_parents:
                forward_path, list_moves_forward = bidirectional_bfs_path(next_position, forward_parents)
                backward_path, list_moves_backward = bidirectional_bfs_path(next_position, backward_parents)
                backward_path.reverse()
                result = forward_path + backward_path
    if result:
        cube.state = forward_path[-1]
        cube = cube.move(list_moves_forward[-1])
        forward_path.append(cube.state)

        list_moves_backward.reverse()
        result = forward_path + backward_path
        list_moves = list_moves_forward + list_moves_backward

    return len(result), len(discovered_states), list_moves

In [7]:
test_num = 1

dict_times_bid_bfs = {}
dict_states_nums_bid_bfs = {}
dict_sol_lens_bid_bfs = {}
dict_mem_bid_bfs = {}

for elem in cube_tests:
    print(f"Execution for case{test_num}")

    tracemalloc.start()
    len_solution, num_of_states, moves = bidirectional_bfs(elem.clone(), unscrambled_cube)
    memory = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    start_time = process_time_ns()
    bidirectional_bfs(elem.clone(), unscrambled_cube)
    end_time = process_time_ns()
    running_time = end_time - start_time

    print(f"Running time: {running_time * 10**(-9)} s")
    print(f"Length of path to solution: {len_solution}")
    print(f"Number of discovered states: {num_of_states}")
    print(f"Used memory: {memory / 2**10} KB")

    dict_times_bid_bfs.update({test_num : running_time * 10**(-9)})
    dict_sol_lens_bid_bfs.update({test_num : len_solution})
    dict_states_nums_bid_bfs.update({test_num : num_of_states})
    dict_mem_bid_bfs.update({test_num : memory / 2**10})
    test_num += 1

    print("\n")

Execution for case1
Running time: 0.046875 s
Length of path to solution: 10
Number of discovered states: 351
Used memory: 259.560546875 KB


Execution for case2
Running time: 0.203125 s
Length of path to solution: 14
Number of discovered states: 2246
Used memory: 1651.623046875 KB


Execution for case3
Running time: 0.140625 s
Length of path to solution: 14
Number of discovered states: 1863
Used memory: 1370.0693359375 KB


Execution for case4
Running time: 1.234375 s
Length of path to solution: 18
Number of discovered states: 14018
Used memory: 11128.212890625 KB


# Bidirectional BFS and A* Comparisons

In [85]:
def plot_results_1(lst, yscale, bar1, bar2, xlabel_txt, ylabel_txt, label1, label2):
    bar_width = 0.25
    plt.subplots(figsize =(7, 5))

    offset_1 = np.arange(len(lst))
    offset_2 = [x + bar_width for x in offset_1] 

    plt.bar(offset_1, bar1, color ='r', width = bar_width, 
        edgecolor ='grey', label =label1) 
    plt.bar(offset_2, bar2, color ='b', width = bar_width, 
        edgecolor ='grey', label =label2) 

    plt.xlabel(xlabel_txt, fontweight ='bold', fontsize = 15) 
    plt.ylabel(ylabel_txt, fontweight ='bold', fontsize = 15) 
    plt.xticks([r + bar_width - 0.07 for r in range(len(lst))], 
        ['case1', 'case2', 'case3', 'case4'])
 
    if yscale == "log":
        plt.yscale("log")
    plt.legend()
    plt.show()

In [86]:
plt.close("all")
plot_results_1([0.001, 0.001, 0.001, 0.001], "log", list(dict_times_astar.values())[:4], list(dict_times_bid_bfs.values())
             , "Time Comparison", "Execution Time (s)", "A*", "Bidirectional BFS")

plot_results_1([3, 3, 3, 3], "-", list(dict_sol_lens_astar.values())[:4], list(dict_sol_lens_bid_bfs.values()),
             "Solution Path Length Comparison", "Path Length", "A*", "Bidirectional BFS")

plot_results_1([0.001, 0.001, 0.001, 0.001], "log", list(dict_states_nums_astar.values())[:4], list(dict_states_nums_bid_bfs.values()),
             "Generated States Comparison", "Number of Discovered States", "A*", "Bidirectional BFS")

plot_results_1([0.001, 0.001, 0.001, 0.001], "log", list(dict_mem_astar.values())[:4], list(dict_mem_bid_bfs.values())
             , "Memory Usage Comparison", "Used Memory (KB)", "A*", "Bidirectional BFS")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Monte Carlo Tree Search Algorithm for Pocket Cube Problem

In [70]:
# The h2 heuristic from homework statement
def manhattan_dist_cube(cube_state) -> int:
    dist = 0
    for elem in range(6):
        for i in range(elem * 4, (elem + 1) * 4):
                dist += abs(cube_state[i] - target[i])
    return dist

def get_available_actions(list_moves : list) -> list:
    avl_moves = [Move.R, Move.Rp, Move.U, Move.Up, Move.F, Move.Fp]
    if len(list_moves) > 0:
        if Move.R == list_moves[-1] or Move.Rp == list_moves[-1]:
            avl_moves.remove(Move.R)
            avl_moves.remove(Move.Rp)
            return avl_moves
        elif Move.U == list_moves[-1] or Move.Up == list_moves[-1]:
            avl_moves.remove(Move.Up)
            avl_moves.remove(Move.U)
            return avl_moves
        elif Move.F == list_moves[-1] or Move.Fp == list_moves[-1]:
            avl_moves.remove(Move.F)
            avl_moves.remove(Move.Fp)
            return avl_moves
    else:
        return avl_moves

def apply_action(state, move_action : Move):
    dummy_cube.state = state
    return dummy_cube.move(move_action).state

def is_final(state):
    if state.tolist() == target:
        return True
    return False

def init_node(parent = None):
    return {N: 0, Q: 0, PARENT: parent, ACTIONS: {}}

def select_action(node, c):

    N_node = node[N]
    max_score = -1
    best_action = None

    for a, n in node[ACTIONS].items():
        crt_score = n[Q] / n[N] + c * sqrt(2 * log(N_node) / n[N])

        if max_score < crt_score:
            max_score = crt_score
            best_action = a

    return best_action

def mcts(state0, budget, h, C):

    tree = init_node()
    list_actions = []
    list_solution_moves = []
    monte_carlo_path = []
    signal = 0

    for x in range(budget):

        node = tree
        state = state0.state
        monte_carlo_path.append(state)

        while (all(action in node[ACTIONS] for action in get_available_actions(list_actions))
            and not is_final(state)
        ):
            new_action = select_action(node, C)
            list_actions.append(new_action)
            list_solution_moves.append(new_action)
            state = apply_action(state, new_action)
            monte_carlo_path.append(state)
            node = node[ACTIONS][new_action]

        if not is_final(state):
            new_action = choice(list(filter(lambda arg: arg not in node[ACTIONS], get_available_actions(list_actions))))
            list_actions.append(new_action)
            list_solution_moves.append(new_action)
            state = apply_action(state, new_action)
            monte_carlo_path.append(state)
            node = init_node(node)
            lst = node[PARENT][ACTIONS]
            lst[new_action] = node

        limit_depth = 0
        max = -1

        while limit_depth < 14 and (not is_final(state)):

            action = choice(get_available_actions(list_actions))
            list_actions.append(action)
            list_solution_moves.append(action)
            state = apply_action(state, action)
            monte_carlo_path.append(state)

            limit_depth += 1
            if h(state) > max:
                max = h(state)
                
            if is_final(state):
                signal = 1
                break

        reward = max
        crt_node = node

        while crt_node:
            crt_node[N] += 1
            crt_node[Q] += reward
            crt_node = crt_node[PARENT]

        if signal == 1:
            break
        else:
            list_solution_moves = []
            monte_carlo_path = []

    if tree:
        final_action = select_action(tree, 0.0)
        return tree[ACTIONS][final_action], len(monte_carlo_path), len(list_actions) + 1, list_solution_moves

In [26]:
dict_times_mcts = {}
dict_states_nums_mcts = {}
dict_sol_lens_mcts = {}

budgets = [1000, 5000, 10000, 20000]
C_lst = [0.1, 0.5]
heuristic_lst = [rotation_cube, manhattan_dist_cube]
max_iter = 20

len_solution = 0
num_of_states = 0
running_time = 0
div_limit = 0

for h in heuristic_lst:
    for C in C_lst:
        for budget in budgets:
            test_num = 1
            for elem in cube_tests:
                for _ in range(max_iter):

                    start_time = process_time_ns()
                    _, len_sol, states_num, moves_mcts = mcts(elem, budget, h, C)
                    end_time = process_time_ns()
                    time = end_time - start_time
                    
                    if len_sol != 0 and states_num != 1:
                        div_limit += 1
                        len_solution += len_sol
                        num_of_states += states_num
                        running_time += time

                if h == heuristic_lst[0]:
                    print(f"Execution for case{test_num}, budget={budget}, C={C}, heuristic=rotation_cube()")
                else:
                    print(f"Execution for case{test_num}, budget={budget}, C={C}, heuristic=manhattan_dist_cube()")

                if div_limit != 0:
                    print(f"Running time: {running_time / div_limit * 10**(-9)} s")
                    print(f"Length of path to solution: {int(len_solution / div_limit)}")
                    print(f"Number of discovered states: {int(num_of_states / div_limit)}")
                    
                    dict_times_mcts.update({(test_num, budget, C, h) : (running_time / div_limit) * 10**(-9)})
                    dict_sol_lens_mcts.update({(test_num, budget, C, h) : int(len_solution / div_limit)})
                    dict_states_nums_mcts.update({(test_num, budget, C, h) : int(num_of_states / div_limit)})

                else:
                    print(f"Running time: {running_time* 10**(-9)} s")
                    print(f"Length of path to solution: {len_solution}")
                    print(f"Number of discovered states: {num_of_states}")

                    dict_times_mcts.update({(test_num, budget, C, h) : running_time * 10**(-9)})
                    dict_sol_lens_mcts.update({(test_num, budget, C, h) : len_solution})
                    dict_states_nums_mcts.update({(test_num, budget, C, h) : num_of_states})

                test_num += 1

                len_solution = 0
                num_of_states = 0
                running_time = 0
                div_limit = 0

                print("\n")

Execution for case1, budget=1000, C=0.1, heuristic=rotation_cube()
Running time: 0.3203125 s
Length of path to solution: 9
Number of discovered states: 5223

Execution for case2, budget=1000, C=0.1, heuristic=rotation_cube()
Running time: 0.15625 s
Length of path to solution: 14
Number of discovered states: 2648

Execution for case3, budget=1000, C=0.1, heuristic=rotation_cube()
Running time: 0.390625 s
Length of path to solution: 20
Number of discovered states: 6461

Execution for case4, budget=1000, C=0.1, heuristic=rotation_cube()
Running time: 0.0 s
Length of path to solution: 0
Number of discovered states: 0

Execution for case1, budget=5000, C=0.1, heuristic=rotation_cube()
Running time: 3.30859375 s
Length of path to solution: 18
Number of discovered states: 56865

Execution for case2, budget=5000, C=0.1, heuristic=rotation_cube()
Running time: 0.015625 s
Length of path to solution: 8
Number of discovered states: 83

Execution for case3, budget=5000, C=0.1, heuristic=rotation_cu

# MCTS Parameters Comparison

In [71]:
def plot_results_2(lst, yscale, bar1, bar2, bar3, bar4, xlabel_txt, ylabel_txt):
    bar_width = 0.21
    plt.subplots(figsize =(10, 6)) 

    offset_1 = np.arange(len(lst))
    offset_2 = [x + bar_width  for x in offset_1]
    offset_3 = [x + bar_width  for x in offset_2] 
    offset_4 = [x + bar_width  for x in offset_3]

    plt.bar(offset_1, bar1, color ='r', width = bar_width, 
        edgecolor ='grey', label ='budget = 1000') 
    plt.bar(offset_2, bar2, color ='y', width = bar_width, 
        edgecolor ='grey', label ='budget = 5000')
    plt.bar(offset_3, bar3, color ='b', width = bar_width, 
        edgecolor ='grey', label ='budget = 10000') 
    plt.bar(offset_4, bar4, color ='g', width = bar_width, 
        edgecolor ='grey', label ='budget = 20000') 

    plt.xlabel(xlabel_txt, fontweight ='bold', fontsize = 15) 
    plt.ylabel(ylabel_txt, fontweight ='bold', fontsize = 15) 
    plt.xticks([r + bar_width + 0.11 for r in range(len(lst))], 
        ['case1', 'case2', 'case3', 'case4'])
 
    if yscale == "log":
        plt.yscale("log")
    plt.legend(loc="upper left")
    plt.show()

In [133]:
def get_info_lists(h, C):

    budgets = [1000, 5000, 10000, 20000]
    info_budgets_times = []
    info_sol_len = []
    info_num_states = []

    for budget in budgets:
        for test in range(1, 5):
            info_budgets_times.append(dict_times_mcts[(test, budget, C, h)])
            info_sol_len.append(dict_sol_lens_mcts[(test, budget, C, h)])
            info_num_states.append(dict_states_nums_mcts[(test, budget, C, h)])

    return info_budgets_times, info_sol_len, info_num_states

In [135]:
# heuristic_lst list contains the memory addresses for both heuristics (rotation_cube and manhattan_dist_cube)
heuristic_lst = [list(dict_times_mcts.keys())[0][3], list(dict_times_mcts.keys())[32][3]]
C_lst = [0.1, 0.5]

plt.close("all")
for h in heuristic_lst:
    for C in C_lst:

            info_budgets_times, info_sol_len, info_num_states = get_info_lists(h, C)
            plot_results_2([0.001, 0.001, 0.001, 0.001], "log",
                       info_budgets_times[:4], 
                       info_budgets_times[4:8],
                       info_budgets_times[8:12],
                       info_budgets_times[12:16],
                            "Time Comparison for Heuristic = "
                       + str(h.__name__) + " and C = "
                       + str(C), "Execution Time (s)")

            plot_results_2([3, 3, 3, 3], "-",
                       info_sol_len[:4], 
                       info_sol_len[4:8],
                       info_sol_len[8:12],
                       info_sol_len[12:16],
             "Solution Path Length Comparison for Heuristic = " + 
                       str(h.__name__) + " and C = "
                       + str(C), "Path Length")

            plot_results_2([0.001, 0.001, 0.001, 0.001], "log",
                       info_num_states[:4], 
                       info_num_states[4:8],
                       info_num_states[8:12],
                       info_num_states[12:16],
             "Generated States Comparison for Heuristic = " + 
                       str(h.__name__) + " and C = "
                       + str(C), "Number of Discovered States")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Best MCTS Parameters, Bidirectional BFS and A* Comparisons

In [74]:
def plot_results_3(lst, yscale, bar1, bar2, bar3, xlabel_txt, ylabel_txt):
    bar_width = 0.21
    plt.subplots(figsize =(10, 6)) 

    offset_1 = np.arange(len(lst))
    offset_2 = [x + bar_width  for x in offset_1]
    offset_3 = [x + bar_width  for x in offset_2] 

    plt.bar(offset_1, bar1, color ='r', width = bar_width, 
        edgecolor ='grey', label ='A*') 
    plt.bar(offset_2, bar2, color ='g', width = bar_width, 
        edgecolor ='grey', label ='Bidirectional BFS')
    plt.bar(offset_3, bar3, color ='b', width = bar_width, 
        edgecolor ='grey', label ='MCTS') 

    plt.xlabel(xlabel_txt, fontweight ='bold', fontsize = 15) 
    plt.ylabel(ylabel_txt, fontweight ='bold', fontsize = 15) 
    plt.xticks([r + bar_width + 0.11 for r in range(len(lst))], 
        ['case1', 'case2', 'case3', 'case4'])
 
    if yscale == "log":
        plt.yscale("log")
    plt.legend(loc="upper left")
    plt.show()

In [46]:
# Best configuration for this MCTS Algorithm
budget = 10000
h = rotation_cube
C = 0.5
test_num = 1
dict_mem_mcts = {}

for elem in cube_tests:
    print(f"Execution for case{test_num}")
    tracemalloc.start()
    _, len_sol, states_num, moves_mcts = mcts(elem, budget, h, C)
    memory = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()
    print(f"Used memory: {memory / 2**10} KB")

    dict_mem_mcts.update({test_num : memory / 2**10})
    test_num += 1

    print("\n")

info_budgets_times, info_sol_len, info_num_states = get_info_lists(list(dict_times_mcts.keys())[0][3], C)
plt.close("all")

plot_results_3([0.001, 0.001, 0.001, 0.001], "log", list(dict_times_astar.values()), list(dict_times_bid_bfs.values()),
               info_budgets_times[8:12]
             , "Time Comparison", "Execution Time (s)")

plot_results_3([3, 3, 3, 3], "-", list(dict_sol_lens_astar.values()), list(dict_sol_lens_bid_bfs.values()),
               info_sol_len[8:12]
             , "Solution Path Length Comparison", "Path Length")

plot_results_3([0.001, 0.001, 0.001, 0.001], "log", list(dict_states_nums_astar.values()), list(dict_states_nums_bid_bfs.values()),
               info_num_states[8:12]
             , "Generated States Comparison", "Number of Discovered States")

plot_results_3([0.001, 0.001, 0.001, 0.001], "log", list(dict_mem_astar.values()), list(dict_mem_bid_bfs.values()),
               list(dict_mem_mcts.values())
             , "Memory Usage Comparison", "Used Memory (KB)")

Execution for case1
Used memory: 3698.2861328125 KB


Execution for case2
Used memory: 5412.2919921875 KB


Execution for case3
Used memory: 5420.5107421875 KB


Execution for case4
Used memory: 5407.8193359375 KB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Pattern Database Solution for Pocket Cube Problem

In [93]:
dist_list = {unscrambled_cube.hash(): (0, "")}

def heuristic_pattern_db():
    queue = [(unscrambled_cube.clone(), 0)]
    while queue:
        curr_state, cost = queue.pop(0)
        for adj_cube in get_neighbours(curr_state).items():
            if cost + 1 < 8:
                if adj_cube[1].hash() not in dist_list:
                    dist_list[adj_cube[1].hash()] = (cost + 1, adj_cube[0])
                    queue.append((adj_cube[1], cost + 1))

def multi_heuristic(cube_state) -> int:
    dummy_cube.state = cube_state
    if dummy_cube.hash() in dist_list:
        return dist_list[dummy_cube.hash()][0]
    else:
        return rotation_cube(cube_state)

tracemalloc.start()
heuristic_pattern_db()
memory_db = tracemalloc.get_traced_memory()[1]
tracemalloc.stop()

dist_list = {unscrambled_cube.hash(): (0, "")}
start_time = process_time_ns()
heuristic_pattern_db()
end_time = process_time_ns()
running_time_db = end_time - start_time

print(f"Running time: {running_time_db * 10**(-9)} s")
print(f"Used memory: {memory_db / 2**10} KB")
print("\n")

Running time: 11.015625 s
Used memory: 29098.1083984375 KB


# A* Algorithm using Pattern Database Technique

In [124]:
test_num = 1

for elem in cube_tests:
    print(f"Execution for case{test_num}")

    tracemalloc.start()
    len_solution, num_of_states, moves_astar_db = astar(elem.clone(), unscrambled_cube, multi_heuristic)
    memory = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    start_time = process_time_ns()
    astar(elem.clone(), unscrambled_cube, multi_heuristic)
    end_time = process_time_ns()
    running_time = end_time - start_time

    print(f"Running time: {running_time * 10**(-9)} s")
    print(f"Length of path to solution: {len_solution}")
    print(f"Number of discovered states: {num_of_states}")
    print(f"Used memory: {memory / 2**10} KB")

    dict_times_astar.update({(test_num, multi_heuristic) : running_time * 10**(-9)})
    dict_sol_lens_astar.update({(test_num, multi_heuristic) : len_solution})
    dict_states_nums_astar.update({(test_num, multi_heuristic) : num_of_states})
    test_num += 1

    print("\n")

Execution for case1
Running time: 0.0 s
Length of path to solution: 6
Number of discovered states: 27
Used memory: 23.0751953125 KB


Execution for case2
Running time: 0.015625 s
Length of path to solution: 8
Number of discovered states: 113
Used memory: 76.0244140625 KB


Execution for case3
Running time: 0.375 s
Length of path to solution: 10
Number of discovered states: 3745
Used memory: 2734.9306640625 KB


Execution for case4
Running time: 4.796875 s
Length of path to solution: 12
Number of discovered states: 42805
Used memory: 33354.33984375 KB


# MCTS Algorithm using Pattern Database Technique

In [ ]:
budgets = [1000, 5000, 10000, 20000]
C_lst = [0.1, 0.5]
max_iter = 20

len_solution = 0
num_of_states = 0
running_time = 0
max_mem = 0
div_limit = 0

for C in C_lst:
    for budget in budgets:
        test_num = 1
        for elem in cube_tests:
            for _ in range(max_iter):

                tracemalloc.start()
                _, len_sol, states_num, moves_mcts = mcts(elem, budget, multi_heuristic, C)
                memory = tracemalloc.get_traced_memory()[1]
                tracemalloc.stop()

                start_time = process_time_ns()
                mcts(elem, budget, multi_heuristic, C)
                end_time = process_time_ns()
                time = end_time - start_time
                    
                if len_sol != 0 and states_num != 1:
                    div_limit += 1
                    len_solution += len_sol
                    num_of_states += states_num
                    running_time += time
                    max_mem += memory

            print(f"Execution for case{test_num}, budget={budget}, C={C}, heuristic=multi_heuristic()")

            if div_limit != 0:
                print(f"Running time: {running_time / div_limit * 10**(-9)} s")
                print(f"Length of path to solution: {int(len_solution / div_limit)}")
                print(f"Number of discovered states: {int(num_of_states / div_limit)}")
                print(f"Used memory: {max_mem / div_limit * 2**10} KB")

                dict_times_mcts.update({(test_num, budget, C, multi_heuristic) : (running_time / div_limit) * 10**(-9)})
                dict_sol_lens_mcts.update({(test_num, budget, C, multi_heuristic) : int(len_solution / div_limit)})
                dict_states_nums_mcts.update({(test_num, budget, C, multi_heuristic) : int(num_of_states / div_limit)})

            else:
                print(f"Running time: {running_time* 10**(-9)} s")
                print(f"Length of path to solution: {len_solution}")
                print(f"Number of discovered states: {num_of_states}")
                print(f"Used memory: {max_mem / 2**10} KB")

                dict_times_mcts.update({(test_num, budget, C, multi_heuristic) : running_time * 10**(-9)})
                dict_sol_lens_mcts.update({(test_num, budget, C, multi_heuristic) : len_solution})
                dict_states_nums_mcts.update({(test_num, budget, C, multi_heuristic) : num_of_states})

            test_num += 1

            len_solution = 0
            num_of_states = 0
            running_time = 0
            div_limit = 0

            print("\n")

# MCTS Parameters Comparison

In [144]:
C_lst = [0.1, 0.5]

plt.close("all")
for C in C_lst:

    info_budgets_times, info_sol_len, info_num_states = get_info_lists(list(dict_times_mcts.keys())[64][3], C)
    plot_results_2([0.001, 0.001, 0.001, 0.001], "log",
                       info_budgets_times[:4], 
                       info_budgets_times[4:8],
                       info_budgets_times[8:12],
                       info_budgets_times[12:16],
                            "Time Comparison for Heuristic = "
                       + str(multi_heuristic.__name__) + " and C = "
                       + str(C), "Execution Time (s)")

    plot_results_2([3, 3, 3, 3], "-",
                       info_sol_len[:4], 
                       info_sol_len[4:8],
                       info_sol_len[8:12],
                       info_sol_len[12:16],
             "Solution Path Length Comparison for Heuristic = " + 
                       str(multi_heuristic.__name__) + " and C = "
                       + str(C), "Path Length")

    plot_results_2([0.001, 0.001, 0.001, 0.001], "log",
                       info_num_states[:4], 
                       info_num_states[4:8],
                       info_num_states[8:12],
                       info_num_states[12:16],
             "Generated States Comparison for Heuristic = " + 
                       str(multi_heuristic.__name__) + " and C = "
                       + str(C), "Number of Discovered States")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Pattern Database MCTS and A* Comparisons

In [ ]:
# Best configuration for this Pattern Database MCTS Algorithm
C = 0.1

info_budgets_times, info_sol_len, info_num_states = get_info_lists(list(dict_times_mcts.keys())[64][3], C)

plt.close("all")
plot_results_1([0.001, 0.001, 0.001, 0.001], "log", list(dict_times_astar.values())[4:], info_budgets_times[:4]
             , "Time Comparison", "Execution Time (s)", "Pattern Database A*", "Pattern Database MCTS")

plot_results_1([3, 3, 3, 3], "-", list(dict_sol_lens_astar.values())[4:], info_sol_len[:4],
             "Solution Path Length Comparison", "Path Length", "Pattern Database A*", "Pattern Database MCTS")

plot_results_1([0.001, 0.001, 0.001, 0.001], "log", list(dict_states_nums_astar.values())[4:], info_num_states[:4],
             "Generated States Comparison", "Number of Discovered States", "Pattern Database A*", "Pattern Database MCTS")